In [1]:
!nvidia-smi

Fri Aug  1 11:06:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.148.08             Driver Version: 570.148.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GH200 480GB             On  |   00000000:DD:00.0 Off |                    0 |
| N/A   66C    P0            563W /  700W |   41132MiB /  97871MiB |     89%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
from gemma_whisper import GemmaWhisperForConditionalGeneration
from transformers import AutoFeatureExtractor, AutoTokenizer, AutoProcessor
import librosa
import torch

torch.set_grad_enabled(False)

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2025-08-01 11:07:00.919712: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754046420.929682  119688 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754046420.934453  119688 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register facto

In [3]:
GemmaWhisperForConditionalGeneration.register_for_auto_class("AutoModel")

In [4]:
!ls gemma3n-audio-whisper-decoder-v3

checkpoint-134000  checkpoint-134500  checkpoint-135000  runs


In [5]:
model = GemmaWhisperForConditionalGeneration.from_pretrained(
    'gemma3n-audio-whisper-decoder-v3/checkpoint-133500', torch_dtype = 'auto')

In [6]:
_ = model.cuda()

In [7]:
feature_extractor = AutoFeatureExtractor.from_pretrained('mesolitica/gemma-3n-e4b-it-audio-encoder')
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3-turbo')
processor = AutoProcessor.from_pretrained('openai/whisper-large-v3-turbo')

In [8]:
from streaming import MDSWriter, LocalDataset

dataset = LocalDataset('mosaic-stt-include-malaysian')

In [11]:
i = -700000
t = dataset[i]['text']
y, sr = librosa.load(dataset[i]['audio_filename'], sr = feature_extractor.sampling_rate)
tokenized = tokenizer(dataset[i]['text'], add_special_tokens = False)['input_ids']
input_ids = []
for k in tokenized:
    input_ids.append(k)
    if k == 50364:
        break
features = feature_extractor([y], return_tensors = 'pt')
features['input_features'] = features['input_features'].cuda()
features['input_features_mask'] = features['input_features_mask'].cuda()
features['attention_mask'] = features['input_features_mask']
features['decoder_input_ids'] = torch.tensor(input_ids)[None].cuda()

generate_kwargs = dict(
    **features,
    max_new_tokens=1024,
    temperature=0.0,
    do_sample=False
)
generation_output = model.generate(**generate_kwargs)
tokenizer.decode(generation_output[0]), t

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


('<|startoftranscript|><|rw|><|transcribe|><|notimestamps|> hakurikijwe ibikenewe kurusha ibindi<|endoftext|>',
 '<|startoftranscript|><|rw|><|transcribe|><|notimestamps|> hakurikijwe ibikenewe kurusha ibindi.<|endoftext|>')

In [12]:
model.push_to_hub('mesolitica/gemma3n-audio-encoder-whisper-decoder')

Processing Files (0 / 0)                : |                                                                                                                                   |  0.00B /  0.00B            
New Data Upload                         : |                                                                                                                                   |  0.00B /  0.00B            

  /tmp/tmpzs73sloo/model.safetensors    :   0%|                                                                                                                               |  215kB / 3.44GB            

Processing Files (0 / 1)                :   0%|                                                                                                                               |  215kB / 3.44GB,   ???B/s  

Processing Files (0 / 1)                :   1%|█▏                                                                                                                             | 32.6M

CommitInfo(commit_url='https://huggingface.co/mesolitica/gemma3n-audio-encoder-whisper-decoder/commit/1af13f83f6365b1e62e8bd66ce1a69d083054f3c', commit_message='Upload GemmaWhisperForConditionalGeneration', commit_description='', oid='1af13f83f6365b1e62e8bd66ce1a69d083054f3c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mesolitica/gemma3n-audio-encoder-whisper-decoder', endpoint='https://huggingface.co', repo_type='model', repo_id='mesolitica/gemma3n-audio-encoder-whisper-decoder'), pr_revision=None, pr_num=None)

In [13]:
y, sr = librosa.load('common_voice_ba_26517811.mp3', sr = feature_extractor.sampling_rate)
input_ids = tokenizer(
    '<|startoftranscript|><|ru|><|transcribe|><|notimestamps|>', 
    add_special_tokens = False, return_tensors = 'pt')['input_ids']
features = feature_extractor([y], return_tensors = 'pt')
features['input_features'] = features['input_features'].cuda()
features['input_features_mask'] = features['input_features_mask'].cuda()
features['attention_mask'] = features['input_features_mask']
features['decoder_input_ids'] = input_ids.cuda()

generate_kwargs = dict(
    **features,
    max_new_tokens=1024,
    temperature=0.1,
    do_sample=True
)
generation_output = model.generate(**generate_kwargs)
tokenizer.decode(generation_output[0])

'<|startoftranscript|><|ru|><|transcribe|><|notimestamps|> Кубы сыра кто был халгя, кешенен, битарафлыгы сепепсе.<|endoftext|>'